<a href="https://colab.research.google.com/github/buklinfur/python_3semester/blob/main/lab_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Семинарские задания


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


torch.cuda.is_available()

True

1. Написать SimpleModel на другом уровне абстракции. Использовать model = nn.Sequential() https://pytorch.org/tutorials/beginner/nn_tutorial.html?highlight=mnist

In [ ]:
input_size = 28 * 28

simple_model = nn.Sequential(
    nn.Flatten(),  # Flattens the 28x28 input images to a 1D tensor of size 784
    nn.Linear(28*28, 128),  # First Linear layer
    nn.ReLU(),  # Activation function
    nn.Linear(128, 10)  # Output layer
)

print (simple_model)

# Usage example
x = torch.randn(64, 1, 28, 28)
output = simple_model(x)
output.shape # The final output is a tensor of shape [batch_size, output_size], where each row corresponds to the logits for the 10 classes

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=128, bias=True)
  (2): ReLU()
  (3): Linear(in_features=128, out_features=10, bias=True)
)


torch.Size([64, 10])

2. С помощью библиотеки torch реализовать модель с прямым проходом, состоящую из 3 полносвязных слоёв с функциями активации: ReLU, tanh, Softmax. Длины векторов на входе 256, на выходе 4, промежуточные: 64 и 16. Использовать модули - nn.Module

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(256, 64)  # Input layer to first hidden layer
        self.fc2 = nn.Linear(64, 16)  # First hidden layer to second hidden layer
        self.fc3 = nn.Linear(16, 4)   # Second hidden layer to output layer

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)  # Apply softmax over the last dimension

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.tanh(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

model = CustomModel ()

# Usage example
x = torch.randn(32, 256)  # Batch of 32 input vectors of size 256
output = model (x)
output.shape

torch.Size([32, 4])

3. Реализовать модель с прямым проходом, состоящую из 2 свёрток (Conv) с функциями активации ReLU и 2 функций MaxPool. Первый слой переводит из 3 каналов в 8, второй из 8 слоёв в 16. На вход подаётся изображение размера 19х19. (19х19x3 -> 18x18x8 -> 9x9x8 -> 8x8x16 -> 4x4x16). Использовать модули - nn.Module

In [ ]:
class ConvModel(nn.Module):
    def __init__(self):
        super(ConvModel, self).__init__()

        self.relu = nn.ReLU()
        # Define the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=2)  # 19x19x3 -> 18x18x8
        self.pool1 = nn.MaxPool2d(kernel_size=2)  # 18x18x8 -> 9x9x8
        # Define the second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=2)  # 9x9x8 -> 8x8x16
        self.pool2 = nn.MaxPool2d(kernel_size=2)  # 8x8x16 -> 4x4x16

    def forward(self, x):
        # First convolutional layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1(x)

        # Second convolutional layer
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool2(x)

        return x

model = ConvModel()
print (model)

# Usage example
x = torch.randn(1, 3, 19, 19)  # Batch of 1, 3 channels, 19x19 pixels
output = model(x)
output.shape # Should be [1, 16, 4, 4]

ConvModel(
  (relu): ReLU()
  (conv1): Conv2d(3, 8, kernel_size=(2, 2), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


torch.Size([1, 16, 4, 4])

4. Объединить сети из п.2 и п.1. На выход изображение размера 19х19, на выходе вектор из 4 элементов

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self):
        super(CombinedModel, self).__init__()

        # From convolutional
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=2),  # 19x19x3 -> 18x18x8
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # 18x18x8 -> 9x9x8
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=2),  # 9x9x8 -> 8x8x16
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)  # 8x8x16 -> 4x4x16
        )

        # From fully-connected
        self.fc_layers = nn.Sequential(
            nn.Linear(16 * 4 * 4, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.Tanh(),
            nn.Linear(16, 4),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten to [batch_size, 16*4*4]
        x = self.fc_layers(x)
        return x

model = CombinedModel()
print (model)

# Usage example
x = torch.randn(1, 3, 19, 19)  # Batch of 1, 3 channels, 19x19 pixels
output = model(x)
output.shape  # Should be [1, 4]

CombinedModel(
  (conv_layers): Sequential(
    (0): Conv2d(3, 8, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=16, bias=True)
    (3): Tanh()
    (4): Linear(in_features=16, out_features=4, bias=True)
    (5): Softmax(dim=1)
  )
)


torch.Size([1, 4])

# Лабораторная работа 13.

1. С помощью библиотеки torch создать модель с прямым проходом, состоящую из 3 слоёв* с функциями активации: ReLu, ReLu, Softmax.
* Два первых слоя могут быть полносвязные или свёрточные на ваш выбор. Последний слой - это FC слой с 10 нейронами.

In [38]:
class Model(nn.Module):
    def __init__(self, use_convolutional=True):
        super(Model, self).__init__()

        self.use_convolutional = use_convolutional

        if use_convolutional:
            self.conv1 = nn.Conv2d(1, 16, kernel_size=3)  # MNIST has 1 channel, 28x28 input -> 26x26
            self.conv2 = nn.Conv2d(16, 32, kernel_size=3)  # 26x26 -> 24x24
            self.fc = nn.Linear(32 * 24 * 24, 10)  # Flattened output to 10 neurons
        else:

            self.fc1 = nn.Linear(28 * 28, 128)  # Flatten 28x28 input to 128
            self.fc2 = nn.Linear(128, 64)  # 128 -> 64
            self.fc3 = nn.Linear(64, 10)  # 64 -> 10

        # Activation functions
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        if self.use_convolutional:
            x = self.conv1(x)
            x = self.relu(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = x.view(x.size(0), -1)  # Flatten for fully connected layer
            x = self.fc(x)
        else:
            x = x.view(x.size(0), -1)  # Flatten input
            x = self.fc1(x)
            x = self.relu(x)
            x = self.fc2(x)
            x = self.relu(x)
            x = self.fc3(x)

        x = self.softmax(x)
        return x

# Usage example
conv_model = Model(use_convolutional=True)
print (conv_model)
fc_model = Model(use_convolutional=False)
print (fc_model)

Model(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc): Linear(in_features=18432, out_features=10, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)
Model(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)


2. Обучить нейросеть распознавать рукописные цифры на датасете MNIST (28х28 px).

In [39]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train = DataLoader(train_dataset, batch_size=64, shuffle=True)
test = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [40]:
import torch.optim as optim

def train_model(model, train_loader, num_epochs=5, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(use_convolutional=True).to(device)

train_model(model, train, num_epochs=5)

Epoch [1/5], Loss: 1.5440
Epoch [2/5], Loss: 1.4869
Epoch [3/5], Loss: 1.4810
Epoch [4/5], Loss: 1.4780
Epoch [5/5], Loss: 1.4754


In [41]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get class predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Evaluate the trained model
evaluate_model(model, test)


Test Accuracy: 98.30%
